In [ ]:
# use python3 for this demonstration of ABC
#!/usr/bin/env/python3
# to activate python3 kernel in jupyter, must use the menu "Kernel" to select the
# python3 kernel
# once the kernel is set, the jupyter doc remembers the last kernel activated.
import numpy as np               # for improved math library
import matplotlib.pyplot as plt  # for plotting
%matplotlib inline

# CW-08 Julia Benchmarking (Python)
## README Item #2
## Lance Clifner, Eric Freda
CS-510
<br>October 18, 2016